In [1]:
%matplotlib inline

import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

D:\lala\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
mnist = input_data.read_data_sets("E:/大三/深度学习/MNIST_data", one_hot=True, reshape=False)

Instructions for updating:
Please use tf.data to implement this functionality.
Extracting E:/大三/深度学习/MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting E:/大三/深度学习/MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting E:/大三/深度学习/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting E:/大三/深度学习/MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
mnist.train.images.shape

(55000, 28, 28, 1)

In [6]:
with tf.Graph().as_default() as g:
    inputs = tf.placeholder(shape=[None, 28, 28, 1], dtype=tf.float32)
    labels = tf.placeholder(shape=[None, 10], dtype=tf.float32)
    
    conv1_weight = tf.get_variable(
        'c1_w', [5, 5, 1, 8], dtype=tf.float32)
    conv1_bias = tf.get_variable(
        'c1_b', [8], dtype=tf.float32)
    conv1_ = tf.nn.conv2d(
        inputs, conv1_weight, [1, 1, 1, 1], 'VALID')
    # shape=[None, 24, 24, 8]
    conv1 = tf.nn.relu(conv1_ + conv1_bias)
    
    # shape=[None, 12, 12, 8]
    pool1 = tf.nn.max_pool(
        conv1, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME')
    
    conv2_weight = tf.get_variable(
        'c2_w', shape=[5, 5, 8, 16], dtype=tf.float32)
    conv2_bias = tf.get_variable(
        'c2_b', shape=[16], dtype=tf.float32)
    conv2_ = tf.nn.conv2d(
        pool1, conv2_weight, [1, 1, 1, 1], 'VALID')
    # shape=[None, 8, 8, 16]
    conv2 = tf.nn.relu(conv2_ + conv2_bias)
    
    # shape=[None, 4, 4, 16]
    pool2 = tf.nn.max_pool(
        conv2, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME')
    
    tmp = tf.reshape(pool2, [-1, 4 * 4 * 16])
    
    logits = tf.keras.layers.Dense(
        10, activation=None)(tmp)
    output = tf.nn.softmax(logits)
    
    saver = tf.train.Saver()
    
    # 代价函数
    loss = tf.reduce_mean(-1 * tf.reduce_sum(
        labels * tf.log(output + 1e-7),
        axis=1))
    
    
    # 正确率
    acc = tf.reduce_mean(
        tf.cast(tf.equal(tf.argmax(labels, axis=1), tf.argmax(output, axis=1)),
                tf.float32))


with tf.Session(graph=g) as sess:
    # 定义梯度下降法优化器
    optim = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    train_op = optim.minimize(loss)
    sess.run(tf.global_variables_initializer())
    
    # 训练模型
    for step in range(20000):
        batch_images, batch_labels = mnist.train.next_batch(32)
        res_loss, _ = sess.run([loss, train_op], feed_dict={
            inputs: batch_images,
            labels: batch_labels
        })
        
        # 输出代价并验证模型
        if step % 500 == 0:
            accs = []
            for test_step in range(10000 // 32):
                batch_images, batch_labels = mnist.test.next_batch(32)
                res_acc = sess.run(acc, feed_dict={
                    inputs: batch_images,
                    labels: batch_labels
                })
                accs.append(res_acc)
            accs = np.mean(accs)
            print('step %5d, loss %2.4f, acc %.4f' % (step, res_loss, accs))
            saver.save(sess, 'conv_ckpt')

step     0, loss 2.2588, acc 0.0605


ValueError: Parent directory of conv_ckpt doesn't exist, can't save.